In [ ]:
import os
from os import path
from astropy.io import fits
import astropy.units as u
from astropy.table import Table
from astropy.constants import G

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('apw-notebook')
%matplotlib inline
import h5py
from sqlalchemy import func
from scipy.optimize import root

from twoface.config import TWOFACE_CACHE_PATH
from twoface.db import (db_connect, AllStar, AllVisit, AllVisitToAllStar, RedClump,
                        StarResult, Status, JokerRun, initialize_db)

from thejoker import JokerParams, TheJoker, JokerSamples
from twoface.sample_prior import make_prior_cache
from twoface.io import load_samples
from twoface.plot import plot_data_orbits
from twobody.celestial import RVOrbit, VelocityTrend1

from scipy.misc import logsumexp

In [ ]:
TWOFACE_CACHE_PATH = path.abspath('../cache/')

In [ ]:
Session, _ = db_connect(path.join(TWOFACE_CACHE_PATH, 'apogee.sqlite'))
session = Session()

In [ ]:
# load the run parameters:
run = session.query(JokerRun).filter(JokerRun.name == 'apogee-jitter').one()

# load the posterior samples:
samples_file = path.join(TWOFACE_CACHE_PATH, '{0}.hdf5'.format(run.name))

In [ ]:
def ln_normal(x, mu, std):
    return -0.5*( ((x-mu) / std)**2 + np.log(2*np.pi*std**2))

# test against (slower) scipy implementation:
from scipy.stats import norm
derp = np.random.uniform(-2, 2, size=100)
pars = np.random.uniform(1E-3, 10, size=2)
assert np.allclose(norm.logpdf(derp, loc=pars[0], scale=pars[1]),
                   ln_normal(derp, *pars))

---

This is just a sanity check that newly generated prior samples look like we expect:

In [ ]:
jitter_kwargs = dict(jitter=(float(run.jitter_mean),
                             float(run.jitter_stddev)),
                     jitter_unit=u.Unit(run.jitter_unit))
print(jitter_kwargs)

params = JokerParams(P_min=8*u.day, P_max=32768*u.day, anomaly_tol=1E-11,
                     **jitter_kwargs)

joker = TheJoker(params)

make_prior_cache('/tmp/jitter.hdf5', joker, 100000, max_batch_size=10000)

with h5py.File('/tmp/jitter.hdf5') as f:
    prior = np.array(f['samples'])
    
    # jitter is saved in km/s
    s = 1000 * prior[:,-1] 
    
plt.hist(np.log(s**2), bins=np.linspace(-8, 18, 32));

Looks OK to me...

---

### Load data by getting particular stars:

In [ ]:
# The aspcapflag bitmask removes STAR_WARN
# The starflag bitmask removes SUSPECT_BROAD_LINES 
stars = session.query(AllStar).join(StarResult, JokerRun, Status)\
                              .filter(Status.id > 0)\
                              .filter(JokerRun.name == 'apogee-jitter')\
                              .filter(AllStar.nvisits > 7)\
                              .filter(AllStar.aspcapflag.op('&')(2**7) == 0)\
                              .filter(AllStar.starflag.op('&')(2**17) == 0)\
                              .all()
#                              .limit(1024).all()
print(len(stars))

In [ ]:
K_n = []
apogee_ids = []
with h5py.File(samples_file) as f:
    # Values that aren't filled get set to nan
    N = len(stars)
    y_nk = np.full((N, 128), np.nan)
    
    for n, key in enumerate([s.apogee_id for s in stars]):
        K = len(f[key]['jitter'])
        
        s = f[key]['jitter'][:] * 1000. # km/s to m/s
        y_nk[n,:K] = np.log(s**2)
        K_n.append(K)
        apogee_ids.append(key)

K_n = np.array(K_n)
apogee_ids = np.array(apogee_ids)

# for nulling out the probability for non-existing samples
mask = np.zeros_like(y_nk)
mask[y_nk == 9999.] = -np.inf

### Load data by getting a random batch of some size:

In [ ]:
# K_n = []
# apogee_ids = []
# with h5py.File(samples_file) as f:
#     # Only load 10000 stars for now
#     N = 2000
#     # N = len(f.keys())
    
#     # Values that aren't filled get set to nan
#     y_nk = np.full((N, 128), np.nan)
    
#     for n,key in enumerate(f):
#         K = len(f[key]['jitter'])
        
#         s = f[key]['jitter'][:] * 1000. # km/s to m/s
#         y_nk[n,:K] = np.log(s**2)
#         K_n.append(K)
#         apogee_ids.append(key)
        
#         if n >= (N-1): 
#             break
            
#         elif n % 1000 == 0:
#             print(n)    

# K_n = np.array(K_n)
# apogee_ids = np.array(apogee_ids)

# # for nulling out the probability for non-existing samples
# mask = np.zeros_like(y_nk)
# mask[y_nk == 9999.] = -np.inf

Re-compute value of the interim prior at the position of the samples

In [ ]:
ln_p0 = ln_normal(y_nk, float(run.jitter_mean), float(run.jitter_stddev))

Check the posterior samples:

In [ ]:
bins = np.linspace(-8, 20, 32)
plt.hist(np.ravel(y_nk)[np.isfinite(np.ravel(y_nk))], bins=bins, normed=True, alpha=0.6, label='all samples');
plt.hist(np.nanmedian(y_nk, axis=1), bins=bins, normed=True, alpha=0.6, label='median over $k$');
plt.legend(loc='upper left', fontsize=16)
plt.xlabel(r'$y = \ln s^2$')

In [ ]:
teff = [s.teff for s in stars]
logg = [s.logg for s in stars]
snr = [s.snr for s in stars]

fig, axes = plt.subplots(2, 2, figsize=(7,7.5), sharex='col', sharey='row')

style = dict(c=np.nanmax(y_nk, axis=1), marker='o', s=8, alpha=0.65,
             vmin=2, vmax=13, linewidth=0)
c = axes[0,0].scatter(teff, snr, **style)
axes[1,0].scatter(teff, logg, **style)
axes[1,1].scatter(snr, logg, **style)

axes[0,0].set_xlim(6000, 3500)
axes[0,0].set_yscale('log')
axes[1,1].set_xscale('log')
axes[1,0].set_ylim(4, 0)

axes[0,0].set_ylabel('SNR')
axes[1,0].set_xlabel('Teff')
axes[1,0].set_ylabel('log$g$')
axes[1,1].set_xlabel('SNR')

fig.tight_layout()

axes[0,1].set_visible(False)

fig.subplots_adjust(left=0.1, right=0.95, top=0.9)

cax = fig.add_axes([0.1, 0.92, 0.85, 0.025])
cb = fig.colorbar(c, cax=cax, orientation='horizontal')
cb.set_label(r'${\rm med}_k\left(y_{nk}\right)$', labelpad=10)
cb.ax.xaxis.tick_top()
cb.ax.xaxis.set_label_position('top')
cb.set_clim(style['vmin'], style['vmax'])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,5))

style = dict(marker='o', s=8, alpha=0.25, linewidth=0)
c = ax.scatter(snr, np.nanmax(y_nk, axis=1), **style)
ax.set_xscale('log')
ax.set_ylim(2, 15)

ax.set_xlabel('SNR')
ax.set_ylabel(r'${\rm med}_k\left(y_{nk}\right)$')

fig.tight_layout()

### The star with the largest value of the smallest $y$ 

In [ ]:
minmax = []
# need the loop because some stars have less than 128 samples
for i, K in zip(range(y_nk.shape[0]), K_n):
    minmax.append(y_nk[i,:K].min())
i = np.argmax(minmax)

print(y_nk[i, :K_n[i]].min(), '{0:.2f} m/s'.format(np.sqrt(np.exp(y_nk[i, :K_n[i]].min()))), apogee_ids[i])
print(i)

star = session.query(AllStar).filter(AllStar.apogee_id == apogee_ids[i]).limit(1).one()
data = star.apogeervdata(clean=True)

samples = JokerSamples(trend_cls=VelocityTrend1, **load_samples(samples_file, apogee_ids[i]))
_ = plot_data_orbits(data, samples)

# residuals?
for j in range(len(samples)):
    this_samples = samples[j]
    
    trend_samples = dict()
    for k in samples.trend_cls.parameters:
        trend_samples[k] = this_samples.pop(k)
    trend = samples.trend_cls(**trend_samples)
    orbit = RVOrbit(trend=trend, **this_samples)
    
    fig, ax = plt.subplots(1,1)
    ax.errorbar(data.t.mjd, (data.rv - orbit.generate_rv_curve(data.t)).to(u.km/u.s).value, 
                data.stddev.to(u.km/u.s).value,
                linestyle='none', marker='o')
    ax.set_ylabel('residuals [{0:latex_inline}]'.format(u.km/u.s))
    ax.set_xlabel('BMJD')
    
    break

### The star with the smallest value of the largest $y$ 

In [ ]:
minmax = []
# need the loop because some stars have less than 128 samples
for i, K in zip(range(y_nk.shape[0]), K_n):
    minmax.append(y_nk[i,:K].max())
i = np.argmin(minmax)

print(y_nk[i, :K_n[i]].min(), '{0:.2f} m/s'.format(np.sqrt(np.exp(y_nk[i, :K_n[i]].min()))), apogee_ids[i])
print(i)

star = session.query(AllStar).filter(AllStar.apogee_id == apogee_ids[i]).limit(1).one()
data = star.apogeervdata()

samples = JokerSamples(trend_cls=VelocityTrend1, **load_samples(samples_file, apogee_ids[i]))
_ = plot_data_orbits(data, samples)

# residuals?
for j in range(len(samples)):
    this_samples = samples[j]
    
    trend_samples = dict()
    for k in samples.trend_cls.parameters:
        trend_samples[k] = this_samples.pop(k)
    trend = samples.trend_cls(**trend_samples)
    orbit = RVOrbit(trend=trend, **this_samples)
    
    fig, ax = plt.subplots(1,1)
    ax.errorbar(data.t.mjd, (data.rv - orbit.generate_rv_curve(data.t)).to(u.km/u.s).value, 
                data.stddev.to(u.km/u.s).value,
                linestyle='none', marker='o')
    ax.set_ylabel('residuals [{0:latex_inline}]'.format(u.km/u.s))
    ax.set_xlabel('BMJD')
    
    break

### Hierarchical inference of jitter parameter distribution

In [ ]:
def ln_likelihood_slow(pars, y, K, ln_prior0):
    mu, std = pars   
    
    ll = np.zeros(y.shape[0])
    for n in range(y.shape[0]):
        delta_ln_prior = ln_normal(y[n, :K[n]], mu, std) - ln_prior0[n, :K[n]]
        ll[n] = logsumexp(delta_ln_prior) - np.log(K[n])
        
    return ll

def ln_likelihood(pars, y, K, ln_prior0):
    """ Original, single Gaussian implementation """
    mu, std = pars      
    delta_ln_prior = ln_normal(y, mu, std) - ln_prior0
    delta_ln_prior[np.isnan(delta_ln_prior)] = -np.inf
    ll = logsumexp(delta_ln_prior, axis=1) - np.log(K)
    return ll

def ln_prob(pars, y, K, ln_prior0):
    # TODO: prior over mu, std of jitter param distribution
    lp = 0.
    
    ll_n = ln_likelihood(pars, y, K, ln_prior0)
    
    if not np.all(np.isfinite(ll_n)):
        return -np.inf
    
    return np.sum(ll_n)

# check that the null masking is working
test_pars = [np.random.uniform(-5, 5),
             np.random.uniform(1E-3, 5)]

assert np.allclose(ln_likelihood_slow(test_pars, y_nk, K_n, ln_p0),
                   ln_likelihood(test_pars, y_nk, K_n, ln_p0))

In [ ]:
# slc = (slice(0,3),) # single
# slc = np.array([512,777])# + list(range(100))) # the two minmax stars above
slc = (slice(None),) # all
# slc = np.array([225, 139])
args = (y_nk[slc], K_n[slc], ln_p0[slc])

(ln_likelihood([-2, 4.], *args),
 ln_likelihood([2, 4.], *args))

In [ ]:
bins = np.linspace(-5, 18, 55)

_n_sub = y_nk[slc].shape[0]
for _n in range(min(_n_sub, 8)):
    plt.hist(y_nk[slc][_n][np.isfinite(y_nk[slc][_n])], bins=bins, 
             alpha=0.5, label='star {0}'.format(_n))

plt.legend(loc='best')
    
vals = np.linspace(bins.min(), bins.max(), 1000)
# lls = ln_normal_mixture(vals, [0.2, 0.8], [0, 1.], [6., 2.])
# plt.plot(vals, np.exp(lls))

In [ ]:
np.std(y_nk[slc].ravel()[y_nk[slc].ravel() < 9999])

In [ ]:
# Single-component likelihood
sigma_y = 2.
# sigma_y = np.std(y_nk[slc].ravel())

lls = []
vals = np.linspace(-5, 15, 128)
for val in vals:
    lls.append(ln_likelihood([val, sigma_y], *args).sum())
    
fig, axes = plt.subplots(1, 2, figsize=(12,5), sharex=True)

axes[0].plot(vals, lls)
axes[0].set_ylabel(r'$\ln p(\{D_n\}|\alpha)$')
axes[1].plot(vals, np.exp(lls - np.max(lls)))
axes[1].set_ylabel(r'$p(\{D_n\}|\alpha)$')

# axes[1].axvline(np.mean(y_nk[slc].ravel()))

axes[0].set_xlabel(r'$\mu_y$')
axes[1].set_xlabel(r'$\mu_y$')

axes[0].xaxis.set_ticks(np.arange(vals.min(), vals.max()+1, 2))

fig.tight_layout()

---

In [ ]:
mu_grid = np.linspace(-10, 10, 27)
# var_grid = np.linspace(0.1, 10, 25)**2
std_grid = np.logspace(-3, 1.5, 25)
mu_grid, std_grid = np.meshgrid(mu_grid, std_grid)

probs = np.array([ln_prob([m, v], *args) 
                  for (m, v) in zip(mu_grid.ravel(), std_grid.ravel())])

In [ ]:
probs.min(), probs.max()

In [ ]:
mu_grid.ravel()[probs.argmax()], std_grid.ravel()[probs.argmax()]

In [ ]:
plt.figure(figsize=(6,5))

plt.pcolormesh(mu_grid, std_grid,
               probs.reshape(mu_grid.shape),
               cmap='Blues', vmin=-1000, vmax=probs.max())
# plt.pcolormesh(mu_grid, std_grid,
#                np.exp(probs.reshape(mu_grid.shape)),
#                cmap='Blues')

plt.yscale('log')
plt.colorbar()
plt.xlabel(r'$\mu_y$')
plt.ylabel(r'$\sigma_y$')

---

In [ ]:
import emcee

In [ ]:
nwalkers = 16
p0 = np.random.normal([5, 4.], [1E-3, 1E-3], size=(nwalkers, 2))

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, dim=2, lnpostfn=ln_prob,
                                args=(y_nk, K_n, ln_p0))
pos,*_ = sampler.run_mcmc(p0, 256)
# sampler.reset()
# _ = sampler.run_mcmc(pos, 512)

In [ ]:
sampler.chain.shape

In [ ]:
for dim in range(sampler.dim):
    plt.figure()
    for walker in sampler.chain[...,dim]:
        plt.plot(walker, marker='', linestyle='-', color='k', 
                 alpha=0.2, drawstyle='steps-mid')